In [ ]:
import qsharp
from IPython.display import Latex
import math

In [2]:
# Ugly python code here

def is_significant(x: float):
    return abs(x) > 1e-9

def is_fractional_part_significant(x: float):
    return is_significant(x - round(x))

def recognize_nice_rational(x: float):
    for denominator in range(1, 31):
        numerator: float = x * float(denominator)
        if not is_fractional_part_significant(numerator):
            return True, int(round(numerator)), denominator
    return False, 0, 1

def get_latex_for_algebraic(numerator: int, denominator: int, root: int, needs_1: bool):
    if denominator != 1:
        needs_1 = True

    latex = ""
    if root == 1:
        if numerator == 1:
            if needs_1:
                latex = 1
        else:
            latex = f"{numerator}"
    else:
        if numerator == 1:
            latex = f"\\sqrt{{{root}}}"
        else:
            latex = f"{numerator} \\sqrt{{{root}}}"

    if denominator != 1:
        latex = f"\\frac{{{latex}}}{{{denominator}}}"
    return latex

def recognize_nice_algebraic(x: float, needs_1: bool):
    is_positive: bool = x > 0
    if not is_positive:
        x = -x
    for root in [1, 2, 3, 5, 6, 7, 10, 11, 13, 14, 15, 17, 19, 21, 22, 23, 26, 29, 30]:
        by_root = x / math.sqrt(root)
        (is_rational, numerator, denominator) = recognize_nice_rational(by_root)
        if is_rational:
            return is_positive, get_latex_for_algebraic(numerator, denominator, root, needs_1)
    return is_positive, f"{x}"

def display_state_latex(dump: qsharp.StateDump):
    dict = dump.get_dict()
    rows = list(dict.items())

    state_latex = ""
    term_number = 0
    for row in rows:
        [state, [real, imag]] = row

        real_significant = is_significant(real)
        imag_significant = is_significant(imag)
        if (not real_significant and not imag_significant):
            continue
        term_number += 1

        if real_significant:
            if imag_significant:
                # both real and imaginary amplitude
                (is_positive1, latex1) = recognize_nice_algebraic(real, True)
                (is_positive2, latex2) = recognize_nice_algebraic(imag, False)
                if not is_positive1:
                    state_latex += "-"
                elif term_number > 1:
                    state_latex += "+"
                imag_sign = "+" if is_positive1 == is_positive2 else "-"
                state_latex += f"\\left( {latex1} {imag_sign} {latex2}i \\right)"
            else:
                # only real amplitude
                (is_positive, latex) = recognize_nice_algebraic(real, False)
                if not is_positive:
                    state_latex += "-"
                elif term_number > 1:
                    state_latex += "+"
                state_latex += f"{latex} \\ "
        else:
            # only imaginary amplitude
            (is_positive, latex) = recognize_nice_algebraic(imag, False)
            if not is_positive:
                state_latex += "-"
            elif term_number > 1:
                state_latex += "+"
            state_latex += f"{latex}i \\ "

        state_latex += f"|\\boldsymbol{{{state}}}\\rangle"
            
    #print(state_latex)
    display(Latex(f"$|\\boldsymbol{{\\psi}}\\rangle = {state_latex}$"))


In [3]:
%%qsharp

use qs = Qubit[2];
H(qs[0]);
CNOT(qs[0], qs[1]);

In [ ]:
dump = qsharp.dump_machine()
display(dump)
display_state_latex(dump)
